In [5]:
import yaml
import pathlib
import time
from Agent.xwAPI import xwBackend

In [2]:
config_file = "./config/config.yaml"
with open(config_file, mode='r') as file:
    config = yaml.load(file, Loader=yaml.Loader)

agent_config = config["Agent"]
with open(agent_config['api_doc_path']) as f:
    api_doc = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
if agent_config['API_backend'] == 'xw':
    xw_backend = xwBackend(agent_config['APP_backend'], api_doc)

In [23]:
path_generator = pathlib.Path(f"../output_dir/intermediate_responses/").glob("**/*")
response_logs = [str(path).split("\\")[-1] for path in path_generator if path.is_file()]
excel_file_names = [f"{log_file[2:-5]}.xlsx" for log_file in response_logs]
excel_file_names

['DemographicProfile.xlsx', 'Dragging.xlsx', 'DemographicProfile.xlsx']

In [25]:
file = f"../dataset/task_sheets/{excel_file_names[0].replace('yaml', 'xlsx')}"
file

'../dataset/task_sheets/DemographicProfile.xlsx'

In [26]:
if file is not None:
    time.sleep(0.5)
    xw_backend.OpenWorkbook(file)

In [27]:
sheet_state = xw_backend.GetSheetsState()
sheet_state

'Sheet state: Sheet "Sheet1" has 4 columns (Headers are A: "Respondents", B: "Sex", C: "Civil Status", D: "Highest Educational Attainment") and 41 rows (1 header row and 40 data rows).'

In [ ]:
def extract_intermediate_responses(log_file):
    intermediate_response = log_file["Success Response"][0]["intermediate response"]
    intermediate_response = [sub_step[:sub_step.find("\nAction API: ")] for sub_step in intermediate_response]
    intermediate_response_dict = {"intermediate response": intermediate_response}
    return intermediate_response_dict

In [ ]:
def save_dictionary(save_path, task_name, dictionary):
    pathlib.Path(f"{save_path}").mkdir(parents=True, exist_ok=True)
    with open(f"{save_path}/{task_name}.yaml", mode="w") as file:
        yaml.dump(dictionary, file, default_flow_style=False)

In [ ]:
def extract_refined_responses(log_file):
    refined_response = log_file["Success Response"][0]["refined response"]
    refined_response = [code_segment[0] for code_segment in refined_response]
    refined_response_dict = {"refined_response": refined_response}
    return refined_response_dict

In [ ]:
for success_task_name in successful_list:
    log_file = load_success_log(success_task_name)
    intermediate_response_dict = extract_intermediate_responses(log_file)
    refined_response_dict = extract_refined_responses(log_file)
    save_dictionary(save_path="../output_dir/intermediate_responses", task_name=success_task_name, dictionary=intermediate_response_dict)
    save_dictionary(save_path="../output_dir/refined_responses", task_name=success_task_name, dictionary=refined_response_dict)